
# Cuaderno de Swaps: Teoría, Fórmulas de Valuación y Implementación en Python

**Contenido**  
1. Teoría: ¿Qué es un swap?  
2. Características principales  
3. Tipos de swaps y fórmulas de valuación  
   - 3.1. **Interest Rate Swap (IRS)** fijo vs. flotante  
   - 3.2. **Overnight Indexed Swap (OIS)**  
   - 3.3. **Basis Swap** (flotante vs. flotante)  
   - 3.4. **Cross-Currency Swap (CCS)** (intercambio de principales y flujos en distintas divisas)  
   - 3.5. (Opcional) **Inflation Swap** – breve mención  
4. Metodologías de valuación (descuento por curvas, tasa par, valuation by legs, OIS discounting, forwards)  
5. Implementación en Python  
6. Ejemplos prácticos


<!-- 
## 1) Teoría: ¿Qué es un swap?
Un **swap** es un contrato derivado en el que dos partes acuerdan **intercambiar** flujos de efectivo futuros según reglas predefinidas. 
Usualmente, los flujos se calculan sobre un **nocional** que no se intercambia (excepto en *cross-currency swaps* o algunos *inflation swaps*).  
El objetivo típico es **gestionar riesgos** (tasa, divisa, inflación), **transformar** la naturaleza de pasivos/activos o **tomar exposición**. -->



## 2) Características principales
- **Nocional**: monto base para calcular intereses.
- **Calendario**: fechas de fijación y pago (frecuencia trimestral, semestral, etc.).
- **Convenciones**: day count (ACT/360, 30/360, ACT/365), business day, *modified following*, etc.
- **Patas (legs)**: cada contraparte paga una “pata”: fija o flotante, o distintas divisas/índices.
- **Tasas de referencia**: SOFR/€STR/TONA (overnight), Ibor/Term SOFR, inflación, etc.
- **Descuento**: uso de **curvas de descuento** (OIS para “risk-free”), y **curvas forward** para proyectar tasas flotantes.
- **Valor razonable (NPV)**: PV(recibido) – PV(pagado). En el *strike* (tasa par), el NPV inicial ≈ 0.



## 3) Tipos de swaps y fórmulas de valuación (resumen)
### 3.1 Interest Rate Swap (IRS) fijo vs. flotante
**Descripción**: una parte paga tasa fija y recibe tasa flotante (o viceversa) sobre el mismo nocional y divisa.

**Fórmula base (leg fija)**  
\[ PV_{fija} = N \cdot \sum_{i=1}^{n} \alpha_i \cdot K \cdot DF(t_i) \]

**Fórmula base (leg flotante)** usando *forward rates* \(L_{i}\):  
\[ PV_{flot} = N \cdot \sum_{i=1}^{n} \alpha_i \cdot L_i \cdot DF(t_i) \]

Aproximación estándar (sin *spread*):  
\[ PV_{flot} \approx N \cdot (DF(t_0) - DF(t_n)) \]

**Tasa par (swap rate)**  
\[ K^{*} = \frac{PV_{flot}}{N \cdot \sum_{i=1}^{n} \alpha_i DF(t_i)} \]



### 3.2 Overnight Indexed Swap (OIS)
**Descripción**: intercambio de una tasa fija contra una tasa **overnight compuesta** (SOFR, €STR, etc.).  

**Leg flotante OIS** (compuesto diariamente):  
\[ (1 + R_{ON}^{comp}) = \prod_{j=1}^{m} (1 + r_{ON,j} \cdot \delta_j) \]
Se descuenta con **curva OIS**.



### 3.3 Basis Swap (flotante vs. flotante)
**Descripción**: intercambio de dos patas flotantes (p. ej., 3M contra 6M) más un **spread** \(s\) en una de las patas.  
\[ PV = N \left( \sum \alpha_i (L_i + s) DF_i - \sum \beta_k M_k DF_k \right) \]
El **spread de equilibrio** \(s^{*}\) hace que \(NPV=0\).



### 3.4 Cross-Currency Swap (CCS)
**Descripción**: intercambio de principales y cupones entre dos divisas. Puede ser fijo–fijo, fijo–flotante, flotante–flotante.  
Valuación por **suma de patas** en su divisa, descontadas con su curva, y convertidas vía FX spot (y, si aplica, *XCCY basis*).  
Ejemplo (simplificado, nocionales \(N_{USD}, N_{MXN}\), spot FX \(S\) en MXN por USD):  
\[ PV = PV_{USD}\cdot S - PV_{MXN} \]



### 3.5 (Opcional) Inflation Swap (mención breve)
Intercambia tasa fija por **inflación realizada** (basada en un índice de precios). La pata de inflación utiliza el cociente de índices entre fechas. 



## 4) Metodologías de valuación
- **Descuento por curvas**: construir (o asumir) curvas de **descuento** y de **forward**. Usualmente: OIS para descuento; forwards desde curvas específicas (p. ej., Term SOFR 1M/3M).
- **Valuación por patas (by legs)**: generar el flujo de cada pata y descontar.
- **Tasa par / swap annuity**: resolver \(K^{*}\) con la *annuity*.
- **Equivalent Floating Leg**: usar \(PV_{flot} pprox N (DF_0 - DF_n)\) cuando aplica.
- **Cross-currency**: valuar cada divisa por separado y convertir usando FX spot y/o forwards; incluir *basis spread* si corresponde.
- **Ajustes**: day count, calendarios, ajustes de negocio, *stub periods*, *amortizing*, *compounding*, etc.



## 5) Implementación en Python
A continuación se presenta una implementación didáctica (no productiva) para:
- Curvas de descuento y forward (sencillas).
- Generación de calendarios (simplificado, sin feriados).
- Pata fija, pata flotante.
- IRS, OIS, Basis Swap y Cross-Currency Swap.
> **Nota**: Se prioriza claridad sobre exhaustividad. Ajuste calendarios/convenciones a necesidad.


In [ ]:

from dataclasses import dataclass
from typing import List
import math
from datetime import date

# ---- Utilidades de día/fecha ----
def year_fraction(d0: date, d1: date, basis: str = "ACT/360") -> float:
    if basis == "ACT/360":
        return (d1 - d0).days / 360.0
    elif basis == "ACT/365":
        return (d1 - d0).days / 365.0
    elif basis == "30/360":
        d0y,d0m,d0d = d0.year, d0.month, min(d0.day, 30)
        d1y,d1m,d1d = d1.year, d1.month, min(d1.day, 30)
        return ((d1y - d0y)*360 + (d1m - d0m)*30 + (d1d - d0d))/360.0
    else:
        raise ValueError("Basis no soportada")
        
def generate_schedule(start: date, end: date, months: int) -> List[date]:
    # Genera fechas de pago regulares (simplificado, sin feriados/business-day roll)
    dates = [start]
    y, m = start.year, start.month
    while True:
        m += months
        y += (m - 1) // 12
        m = (m - 1) % 12 + 1
        d = min(start.day, 28)  # simplificación: fijar al <=28
        nd = date(y, m, d)
        dates.append(nd)
        if nd >= end:
            break
    if dates[-1] != end:
        dates[-1] = end
    return dates


In [ ]:

from dataclasses import dataclass
from typing import List

@dataclass
class DiscountCurve:
    """Curva de descuento simple con interpolación log-lineal en tasas zero."""
    pillars: List[float]      # tiempos (años)
    dfs: List[float]          # discount factors en los pilares
    
    def df(self, t: float) -> float:
        if t <= 0:
            return 1.0
        for T, D in zip(self.pillars, self.dfs):
            if abs(T - t) < 1e-12:
                return D
        if t <= self.pillars[0]:
            z0 = -math.log(self.dfs[0]) / max(self.pillars[0], 1e-12)
            return math.exp(-z0 * t)
        for i in range(1, len(self.pillars)):
            if t <= self.pillars[i]:
                T0, D0 = self.pillars[i-1], self.dfs[i-1]
                T1, D1 = self.pillars[i], self.dfs[i]
                z0 = -math.log(D0)/max(T0,1e-12)
                z1 = -math.log(D1)/max(T1,1e-12)
                w = (t - T0)/(T1 - T0)
                zt = z0*(1-w) + z1*w
                return math.exp(-zt * t)
        Tn, Dn = self.pillars[-1], self.dfs[-1]
        zn = -math.log(Dn)/max(Tn,1e-12)
        return math.exp(-zn * t)


In [ ]:

from dataclasses import dataclass

@dataclass
class ForwardCurve:
    """Forward curve derivada de la curva de descuento (marco de 1 curva)."""
    disc: DiscountCurve
    
    def fwd_simple(self, t0: float, t1: float) -> float:
        if t1 <= t0:
            return 0.0
        df0 = self.disc.df(t0)
        df1 = self.disc.df(t1)
        tau = t1 - t0
        return (df0/df1 - 1.0) / tau


In [ ]:

from dataclasses import dataclass
from typing import List
from datetime import date

@dataclass
class FixedLeg:
    notional: float
    pay_dates: List[date]
    accrual_basis: str
    fixed_rate: float  # K
    
    def pv(self, val_date: date, disc: DiscountCurve) -> float:
        pv = 0.0
        for i in range(1, len(self.pay_dates)):
            t0, t1 = self.pay_dates[i-1], self.pay_dates[i]
            if t1 <= val_date:
                continue
            tau = year_fraction(t0, t1, self.accrual_basis)
            T = year_fraction(val_date, t1, "ACT/365")
            df = disc.df(T)
            pv += self.notional * tau * self.fixed_rate * df
        return pv


In [ ]:

from dataclasses import dataclass
from typing import List
from datetime import date

@dataclass
class FloatLeg:
    notional: float
    fix_dates: List[date]      # resets (inicio del período)
    pay_dates: List[date]      # pagos (fin del período)
    accrual_basis: str
    fwd_curve: ForwardCurve
    spread: float = 0.0        # para basis swaps
    
    def pv(self, val_date: date, disc: DiscountCurve) -> float:
        pv = 0.0
        for i in range(1, len(self.pay_dates)):
            t0, t1 = self.fix_dates[i-1], self.pay_dates[i]
            if t1 <= val_date:
                continue
            tau = year_fraction(self.fix_dates[i-1], self.pay_dates[i], self.accrual_basis)
            Tpay = year_fraction(val_date, self.pay_dates[i], "ACT/365")
            t_start = year_fraction(val_date, self.fix_dates[i-1], "ACT/365")
            t_end   = year_fraction(val_date, self.pay_dates[i], "ACT/365")
            fwd = self.fwd_curve.fwd_simple(max(t_start,0.0), max(t_end,0.0))
            df = disc.df(Tpay)
            pv += self.notional * tau * (fwd + self.spread) * df
        return pv


In [ ]:

from dataclasses import dataclass

@dataclass
class InterestRateSwap:
    pay_fixed: bool
    fixed_leg: FixedLeg
    float_leg: FloatLeg
    
    def npv(self, val_date: date, disc: DiscountCurve) -> float:
        pv_fixed = self.fixed_leg.pv(val_date, disc)
        pv_float = self.float_leg.pv(val_date, disc)
        return (pv_fixed - pv_float) if self.pay_fixed else (pv_float - pv_fixed)
    
    def par_rate(self, val_date: date, disc: DiscountCurve) -> float:
        annuity = 0.0
        for i in range(1, len(self.fixed_leg.pay_dates)):
            t0, t1 = self.fixed_leg.pay_dates[i-1], self.fixed_leg.pay_dates[i]
            if t1 <= val_date:
                continue
            tau = year_fraction(t0, t1, self.fixed_leg.accrual_basis)
            T = year_fraction(val_date, t1, "ACT/365")
            annuity += tau * disc.df(T)
        pv_float = self.float_leg.pv(val_date, disc)
        return pv_float / max(self.fixed_leg.notional * annuity, 1e-12)


In [ ]:

from dataclasses import dataclass

@dataclass
class OISSwap:
    notional: float
    fixed_rate: float
    pay_dates: List[date]
    accrual_basis: str
    disc: DiscountCurve
    fwd_curve: ForwardCurve
    
    def pv(self, val_date: date) -> float:
        pv_fixed = 0.0
        pv_float = 0.0
        for i in range(1, len(self.pay_dates)):
            t0, t1 = self.pay_dates[i-1], self.pay_dates[i]
            if t1 <= val_date:
                continue
            tau = year_fraction(t0, t1, self.accrual_basis)
            Tpay = year_fraction(val_date, t1, "ACT/365")
            df = self.disc.df(Tpay)
            t_start = year_fraction(val_date, t0, "ACT/365")
            t_end   = year_fraction(val_date, t1, "ACT/365")
            r_equiv = self.fwd_curve.fwd_simple(max(t_start,0.0), max(t_end,0.0))
            pv_fixed += self.notional * tau * self.fixed_rate * df
            pv_float += self.notional * tau * r_equiv * df
        return pv_float - pv_fixed  # receiver-float/payer-fixed


In [ ]:

from dataclasses import dataclass

@dataclass
class BasisSwap:
    notional: float
    legA: FloatLeg  # con spread 's' si aplica
    legB: FloatLeg
    
    def npv(self, val_date: date, disc: DiscountCurve) -> float:
        return self.legA.pv(val_date, disc) - self.legB.pv(val_date, disc)
    
    def par_spread(self, val_date: date, disc: DiscountCurve) -> float:
        denom = 0.0
        for i in range(1, len(self.legA.pay_dates)):
            t0, t1 = self.legA.fix_dates[i-1], self.legA.pay_dates[i]
            if t1 <= val_date:
                continue
            tau = year_fraction(t0, t1, self.legA.accrual_basis)
            T = year_fraction(val_date, t1, "ACT/365")
            denom += tau * disc.df(T)
        base_npv = self.legB.pv(val_date, disc) - self.legA.pv(val_date, disc)
        return base_npv / max(self.notional * denom, 1e-12)


In [ ]:

from dataclasses import dataclass

@dataclass
class CrossCurrencySwap:
    notional_foreign: float     # p.ej., USD
    notional_domestic: float    # p.ej., MXN
    pay_dates_foreign: List[date]
    pay_dates_domestic: List[date]
    basis_foreign: str
    basis_domestic: str
    fwd_foreign: ForwardCurve
    fwd_domestic: ForwardCurve
    disc_foreign: DiscountCurve
    disc_domestic: DiscountCurve
    fx_spot: float              # doméstica por 1 extranjera (MXN por USD)
    spread_foreign: float = 0.0
    spread_domestic: float = 0.0
    exchange_principals: bool = True
    
    def pv_domestic(self, val_date: date) -> float:
        # PV de la pata extranjera (en su divisa), convertida a doméstica, menos PV de la doméstica.
        pv_for = 0.0
        for i in range(1, len(self.pay_dates_foreign)):
            t0, t1 = self.pay_dates_foreign[i-1], self.pay_dates_foreign[i]
            if t1 <= val_date:
                continue
            tau = year_fraction(t0, t1, self.basis_foreign)
            Tpay = year_fraction(val_date, t1, "ACT/365")
            df_for = self.disc_foreign.df(Tpay)
            t_start = year_fraction(val_date, t0, "ACT/365")
            t_end   = year_fraction(val_date, t1, "ACT/365")
            fwd = self.fwd_foreign.fwd_simple(max(t_start,0.0), max(t_end,0.0))
            pv_for += self.notional_foreign * tau * (fwd + self.spread_foreign) * df_for
        if self.exchange_principals:
            T_end_for = year_fraction(val_date, self.pay_dates_foreign[-1], "ACT/365")
            pv_for += self.notional_foreign * self.disc_foreign.df(T_end_for)  # principal back
        
        pv_for_dom = pv_for * self.fx_spot
        
        pv_dom = 0.0
        for i in range(1, len(self.pay_dates_domestic)):
            t0, t1 = self.pay_dates_domestic[i-1], self.pay_dates_domestic[i]
            if t1 <= val_date:
                continue
            tau = year_fraction(t0, t1, self.basis_domestic)
            Tpay = year_fraction(val_date, t1, "ACT/365")
            df_dom = self.disc_domestic.df(Tpay)
            t_start = year_fraction(val_date, t0, "ACT/365")
            t_end   = year_fraction(val_date, t1, "ACT/365")
            fwd = self.fwd_domestic.fwd_simple(max(t_start,0.0), max(t_end,0.0))
            pv_dom += self.notional_domestic * tau * (fwd + self.spread_domestic) * df_dom
        if self.exchange_principals:
            T_end_dom = year_fraction(val_date, self.pay_dates_domestic[-1], "ACT/365")
            pv_dom += self.notional_domestic * self.disc_domestic.df(T_end_dom)
        
        return pv_for_dom - pv_dom



## 6) Ejemplos prácticos
A continuación construimos curvas simples, generamos calendarios y valuamos cada tipo de swap.


In [ ]:

# 6.1 Curvas de ejemplo (muy simplificadas)
pillars = [0.5, 1.0, 2.0, 3.0, 5.0]
dfs_usd = [0.99, 0.97, 0.94, 0.90, 0.85]   # ejemplo
dfs_mxn = [0.98, 0.95, 0.90, 0.85, 0.78]   # ejemplo

disc_usd = DiscountCurve(pillars, dfs_usd)
disc_mxn = DiscountCurve(pillars, dfs_mxn)

fwd_usd = ForwardCurve(disc_usd)
fwd_mxn = ForwardCurve(disc_mxn)

from datetime import date
val_date = date(2025, 8, 31)
start = date(2025, 8, 31)
end_2y = date(2027, 8, 31)

sched_semi = generate_schedule(start, end_2y, months=6)   # semestral
sched_quarter = generate_schedule(start, end_2y, months=3)  # trimestral


In [ ]:

# 6.2 IRS fijo vs flotante
fixed_leg = FixedLeg(
    notional=100_000_000,
    pay_dates=sched_semi,
    accrual_basis="30/360",
    fixed_rate=0.05
)

float_leg = FloatLeg(
    notional=100_000_000,
    fix_dates=sched_semi[:-1],
    pay_dates=sched_semi,
    accrual_basis="ACT/360",
    fwd_curve=fwd_usd
)

irs = InterestRateSwap(pay_fixed=True, fixed_leg=fixed_leg, float_leg=float_leg)
npv_irs = irs.npv(val_date, disc_usd)
par_rate_irs = irs.par_rate(val_date, disc_usd)
(npv_irs, par_rate_irs)


In [ ]:

# 6.3 OIS (aprox) – fijo vs overnight compuesto equivalente
ois = OISSwap(
    notional=100_000_000,
    fixed_rate=0.045,
    pay_dates=sched_quarter,
    accrual_basis="ACT/360",
    disc=disc_usd,
    fwd_curve=fwd_usd
)
npv_ois = ois.pv(val_date)
npv_ois


In [ ]:

# 6.4 Basis Swap: 3M + s vs 6M
leg3m = FloatLeg(
    notional=100_000_000,
    fix_dates=sched_quarter[:-1],
    pay_dates=sched_quarter,
    accrual_basis="ACT/360",
    fwd_curve=fwd_usd,
    spread=0.0  # s a resolver
)

leg6m = FloatLeg(
    notional=100_000_000,
    fix_dates=sched_semi[:-1],
    pay_dates=sched_semi,
    accrual_basis="ACT/360",
    fwd_curve=fwd_usd,
)

basis = BasisSwap(notional=100_000_000, legA=leg3m, legB=leg6m)
s_par = basis.par_spread(val_date, disc_usd)
s_par


In [ ]:

# 6.5 Cross-Currency Swap (simplificado): USD vs MXN, flotante-flotante con intercambio de principales
fx_spot = 18.0  # MXN por USD (ejemplo)
ccs = CrossCurrencySwap(
    notional_foreign=10_000_000,    # USD
    notional_domestic=10_000_000*fx_spot,  # MXN
    pay_dates_foreign=sched_semi,
    pay_dates_domestic=sched_quarter,
    basis_foreign="ACT/360",
    basis_domestic="ACT/360",
    fwd_foreign=fwd_usd,
    fwd_domestic=fwd_mxn,
    disc_foreign=disc_usd,
    disc_domestic=disc_mxn,
    fx_spot=fx_spot,
    spread_foreign=0.0,
    spread_domestic=0.0020,  # 20 bps en MXN, por ejemplo
    exchange_principals=True
)
npv_ccs_dom = ccs.pv_domestic(val_date)
npv_ccs_dom



### Notas finales
- Para un entorno productivo, se deben incorporar calendarios reales, feriados, ajustes *business day*, regímenes de capitalización, *stubs*, interpolaciones robustas, construcción de curvas multi-índice (OIS vs forwards), y *curve bootstrapping*.
- Para CCS con *basis*, normalmente se modela mediante curvas con *basis spread* o ajustes en las forwards/discounts.
- La aproximación de \(PV_{flot} pprox N (DF_0 - DF_n)\) es útil en IRS libres de *spread* cuando la curva de descuento y la de proyección coinciden.
